# I.XGBoost Regressor

## Triển khai và đào tạo mô hình XGBoost.
Mục tiêu chính của chúng ta là sử dụng mô hình này để giải quyết một bài toán regresion, cụ thể là dự đoán diện tích khu vực bị cháy trong rừng.

## 1.load libraries

In [36]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error , mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

## 2.download data

In [3]:
# download data
%cd /content/drive/MyDrive/Colab Notebooks/MODULE03/data
! gdown '1xwJmYJxEia06sxUdJyGO7JFx4DNK1fbp'

/content/drive/MyDrive/Colab Notebooks/MODULE03/data
Downloading...
From: https://drive.google.com/uc?id=1xwJmYJxEia06sxUdJyGO7JFx4DNK1fbp
To: /content/drive/MyDrive/Colab Notebooks/MODULE03/data/Problem3.csv
100% 37.4k/37.4k [00:00<00:00, 56.5MB/s]


In [4]:
%cd ..

/content/drive/MyDrive/Colab Notebooks/MODULE03


## 3.load dataset

In [37]:
# load dataset
dataset_path = '/content/drive/MyDrive/Colab Notebooks/MODULE03/data/Problem3.csv'
data_df = pd.read_csv ( dataset_path )
data_df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,4.468204,26.2,94.3,1.808289,8.2,51,6.7,False,0.000000
1,7,4,oct,tue,4.517431,35.4,669.1,2.041220,18.0,33,0.9,False,0.000000
2,7,4,oct,sat,4.517431,43.7,686.9,2.041220,14.6,33,1.3,False,0.000000
3,8,6,mar,fri,4.529368,33.3,77.5,2.302585,8.3,97,4.0,True,0.000000
4,8,6,mar,sun,4.503137,51.3,102.2,2.360854,11.4,99,1.8,False,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4,3,aug,sun,4.414010,56.7,665.6,1.064711,27.8,32,2.7,False,2.006871
506,2,4,aug,sun,4.414010,56.7,665.6,1.064711,21.9,71,5.8,False,4.012592
507,7,4,aug,sun,4.414010,56.7,665.6,1.064711,21.2,70,6.7,False,2.498152
508,1,4,aug,sat,4.558079,146.0,614.7,2.509599,25.6,42,4.0,False,0.000000


## 4.filter all column is not num

In [38]:
# filter all column is not num
categorical_cols = data_df.select_dtypes(include =['object','bool']).columns.to_list()
categorical_cols

['month', 'day', 'rain']

In [39]:
for col_name in categorical_cols :
  n_categories = data_df[col_name].nunique()
  print (f'Number of categories in {col_name}:{n_categories}')

Number of categories in month:12
Number of categories in day:7
Number of categories in rain:2


## 5.encoded all column not num

In [40]:
# encoded all column not num
ordinal_encoder = OrdinalEncoder()
encoded_categorical_cols = ordinal_encoder.fit_transform(data_df[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical_cols,columns = categorical_cols)
numerical_df = data_df.drop(categorical_cols, axis =1)
encoded_df = pd.concat([numerical_df, encoded_categorical_df], axis =1)

In [41]:
# Bộ dữ liệu sau khi đã chuyển đổi toàn bộ các đặc trưng Categorical về dạng số
encoded_df

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,area,month,day,rain
0,7,5,4.468204,26.2,94.3,1.808289,8.2,51,6.7,0.000000,7.0,0.0,0.0
1,7,4,4.517431,35.4,669.1,2.041220,18.0,33,0.9,0.000000,10.0,5.0,0.0
2,7,4,4.517431,43.7,686.9,2.041220,14.6,33,1.3,0.000000,10.0,2.0,0.0
3,8,6,4.529368,33.3,77.5,2.302585,8.3,97,4.0,0.000000,7.0,0.0,1.0
4,8,6,4.503137,51.3,102.2,2.360854,11.4,99,1.8,0.000000,7.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,4,3,4.414010,56.7,665.6,1.064711,27.8,32,2.7,2.006871,1.0,3.0,0.0
506,2,4,4.414010,56.7,665.6,1.064711,21.9,71,5.8,4.012592,1.0,3.0,0.0
507,7,4,4.414010,56.7,665.6,1.064711,21.2,70,6.7,2.498152,1.0,3.0,0.0
508,1,4,4.558079,146.0,614.7,2.509599,25.6,42,4.0,0.000000,1.0,2.0,0.0


## 6.Tách các column thành 2 biến X,Y cho bài toán supervised learning

In [42]:
# Tách các column thành 2 biến X,Y cho bài toán supervised learning
'''Trong bài toán này target/label là ’area’ diện tích cháy rừng'''
X = encoded_df.drop(columns=['area'])
y = encoded_df['area']

In [24]:
X

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,month,day,rain
0,7,5,4.468204,26.2,94.3,1.808289,8.2,51,6.7,7.0,0.0,0.0
1,7,4,4.517431,35.4,669.1,2.041220,18.0,33,0.9,10.0,5.0,0.0
2,7,4,4.517431,43.7,686.9,2.041220,14.6,33,1.3,10.0,2.0,0.0
3,8,6,4.529368,33.3,77.5,2.302585,8.3,97,4.0,7.0,0.0,1.0
4,8,6,4.503137,51.3,102.2,2.360854,11.4,99,1.8,7.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
505,4,3,4.414010,56.7,665.6,1.064711,27.8,32,2.7,1.0,3.0,0.0
506,2,4,4.414010,56.7,665.6,1.064711,21.9,71,5.8,1.0,3.0,0.0
507,7,4,4.414010,56.7,665.6,1.064711,21.2,70,6.7,1.0,3.0,0.0
508,1,4,4.558079,146.0,614.7,2.509599,25.6,42,4.0,1.0,2.0,0.0


In [28]:
y

,area
0,0.000000
1,0.000000
2,0.000000
3,0.000000
4,0.000000
...,...
505,2.006871
506,4.012592
507,2.498152
508,0.000000


## 7.Chia tập dữ liệu huấn luyện

In [43]:
# Chia tập dữ liệu huấn luyện
'''
Dựa vào bộ dữ liệu gốc, ta cần chia thành hai tập dữ liệu con,
một dùng cho việc huấn luyện mô hình và một cho việc đánh giá mô hình.
Ở đây, ta sẽ chia theo tỷ lệ 7:3 và tham số ngẫu nhiên random_state = 7
'''
X_train , X_test , y_train , y_test = train_test_split (X, y, test_size =0.3, random_state =7)

## 8.Huấn luyện mô hình với tập data train

In [44]:
# Huấn luyện mô hình với tập data train
'''
Chúng ta xây dựng model XGBoost cho Regression task với các than số sau:
seed=7, learning_rate=0.01, n_estimators=102, max_depth=3.
Sau đó ta sẽ huấn luyện mô hình với tập data train
'''
xg_reg = xgb.XGBRegressor( seed =7, learning_rate =0.01, n_estimators =102, max_depth =3)
xg_reg.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=102, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## 9.Dự đoán trên tập data test

In [45]:
# Sử dụng mô hình đã huấn luyện để thực hiện dự đoán trên tập data test
preds = xg_reg.predict(X_test)

In [46]:
preds

array([1.0842953 , 1.1191528 , 0.8486002 , 2.0224624 , 1.2788543 ,
       1.2250828 , 1.3104433 , 1.2034497 , 1.049782  , 1.1554059 ,
       1.199015  , 1.1923933 , 1.0917312 , 1.0008674 , 1.0240752 ,
       1.2901965 , 0.8638762 , 1.0937988 , 0.99773127, 1.1649472 ,
       1.1940109 , 1.3383218 , 1.0232271 , 1.2133856 , 1.1919765 ,
       1.1378535 , 1.0194002 , 1.0811108 , 1.184996  , 1.3049333 ,
       1.1830077 , 1.3067408 , 1.1667005 , 1.3425988 , 1.3937967 ,
       1.0007488 , 1.208391  , 1.2034497 , 1.0767705 , 1.2819778 ,
       1.1705064 , 1.2479149 , 0.92552763, 1.8053247 , 1.0428364 ,
       1.0156305 , 1.1495866 , 1.1290253 , 1.026823  , 1.174421  ,
       1.0199474 , 1.1087017 , 1.3341991 , 1.292637  , 1.3510199 ,
       1.0883718 , 0.9601164 , 1.2643764 , 1.117172  , 1.3104433 ,
       1.1043178 , 1.1449616 , 1.0224761 , 1.1609155 , 1.2342656 ,
       1.1611481 , 1.4087317 , 1.0568017 , 0.9867578 , 1.0175698 ,
       0.9044889 , 1.1912946 , 1.3288516 , 1.2732868 , 1.06959

## 10.Đánh Giá mô hình
Để thực hiện đánh giá mô hình trong bài toán regression thì Mean Absolute Error (MAE) và Meanquared Error (MSE) là hai trong số những độ đo phổ biến nhất được sử dụng:

In [47]:
'''
Mean Absolute Error (MAE): Độ đo này cho chúng ta biết trung bình sự chênh lệch
giữa giá trị dự đoán và giá trị thực tế. Nó được tính bằng cách lấy trung bình của trị tuyệt
đối của sự khác biệt giữa giá trị dự đoán và giá trị thực sự.
'''
mae = mean_absolute_error(y_test, preds)
print("Mean Absolute Error (MAE):", mae)

Mean Absolute Error (MAE): 1.1484401341167767


In [49]:
'''
Mean Squared Error (MSE): Tương tự như MAE, nhưng thay vì lấy trị tuyệt đối của
sự khác biệt, MSE sẽ lấy bình phương của sự khác biệt. Điều này có nghĩa là các giá trị dự
đoán xa giá trị thực tế sẽ bị trừng phạt nặng hơn hơn.
'''
mse = mean_squared_error(y_test, preds)
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 1.8845074196256495


# II.XGBoost Classifier

## Triển khai và đào tạo mô hình XGBoost.
Mục tiêu chính của chúng ta là sử dụng mô hình này để giải
quyết một bài toán classification, cụ thể là dự đoán chất lượng của wine

## 1.load libraries

In [50]:
# load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## 2.download data

In [51]:
%cd /content/drive/MyDrive/Colab Notebooks/MODULE03/data
! gdown '1pVdH-2b_odeuEPdXbLQYDcHXxgqqBK4i'

/content/drive/MyDrive/Colab Notebooks/MODULE03/data
Downloading...
From: https://drive.google.com/uc?id=1pVdH-2b_odeuEPdXbLQYDcHXxgqqBK4i
To: /content/drive/MyDrive/Colab Notebooks/MODULE03/data/Problem4.csv
100% 12.3k/12.3k [00:00<00:00, 37.8MB/s]


## 3.load dataset

In [52]:
%cd ..

/content/drive/MyDrive/Colab Notebooks/MODULE03


In [53]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/MODULE03/data/Problem4.csv'
data_df = pd.read_csv(dataset_path)

In [55]:
#mẫu dữ liệu trong data
data_df

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,Target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0,2
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0,2
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0,2


## 4.filter all column is not num

In [59]:
# filter all column is not num
categorical_cols = data_df.select_dtypes(include =['object','bool']).columns.to_list()
categorical_cols


[]

## 5.encoded all column not num
Trong bộ dữ liệu chỉ chứa các đặc trưng dạng số và không có bất kỳ đặc trưng dạng danh mục (categorical) nào, thì không cần thiết phải thực hiện bước mã hóa (encoding)

## 6.Tách các column thành 2 biến X,Y cho bài toán supervised learning

Trong bài toán dự đoán này, chúng ta quan tâm đến việc dự đoán chất lượng của rượu vang (loại 0, 1, 2) dựa trên một loạt các thuộc tính được cung cấp. Loại chất lượng của rượu vang, được lưu trữ trong cột "Target", ta sẽ gán giá trị từ cột này vào biến y (là cột cuối cùng). Những thuộc tính còn lại sẽ được gán vào biến X

In [60]:
# Tách các column thành 2 biến X,Y cho bài toán supervised learning
X, y = data_df.iloc[: , : -1],data_df.iloc[: , -1]

In [61]:
X

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [62]:
y

,Target
0,0
1,0
2,0
3,0
4,0
...,...
173,2
174,2
175,2
176,2


## 7.Chia tập dữ liệu huấn luyện

Dựa vào bộ dữ liệu gốc, ta cần chia thành hai tập dữ liệu con, một dùng cho việc huấn luyện mô hình và một cho việc đánh giá mô hình. Ở đây, ta sẽ chia theo tỷ lệ 7:3 và tham số ngẫu nhiên random_state = 7

In [64]:
# Chia tập dữ liệu huấn luyện
X_train , X_test , y_train , y_test = train_test_split (X, y, test_size =0.3, random_state =7)


## 8.Huấn luyện mô hình với tập data train
Chúng ta xây dựng model XGBoost cho Classification task với các tham số sau seed=7. Sau đó ta sẽ huấn luyện mô hình với tập data train

In [66]:
# Huấn luyện mô hình với tập data train
xg_class = xgb.XGBClassifier(seed =7)
xg_class.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

## 9.Dự đoán trên tập data test

Để đảm bảo mô hình của chúng ta không chỉ hoạt động tốt trên dữ liệu huấn luyện mà còn có khả năng tổng quát hóa tốt trên dữ liệu mới, việc sử dụng một tập test là rất quan trọng. Tập test này sẽ chứa một phần dữ liệu mà mô hình chưa từng thấy trong quá trình huấn
luyện

In [67]:
# Dự đoán mô hình sau khi huấn luyện
preds = xg_class.predict( X_test )

In [68]:
preds

array([2, 0, 2, 2, 1, 2, 1, 0, 1, 2, 0, 1, 2, 1, 1, 1, 1, 2, 0, 0, 1, 1,
       1, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 1, 1, 2, 2, 0, 2, 0, 1, 2, 2,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 0])

## 10.Đánh Giá mô hình
Đối với bài toán phân loại (Classification), việc đánh giá hiệu suất mô hình thường sử dụng các độ đo khác nhau so với bài toán hồi quy (Regression). Trong bài toán phân loại, "accuracy" là một trong những độ đo cơ bản và phổ biến nhất.

Accuracy (Độ chính xác): Độ chính xác cho biết tỷ lệ mẫu được phân loại đúng trong tất cả các mẫu của tập dữ liệu. Nó được tính bằng cách lấy số mẫu được phân loại đúng chia cho tổng số mẫu.

### Accuracy = (Số mẫu phân loại đúng)/( Tổng số mẫu)

In [69]:
# tính Accuracy để đánh giá mô hình
train_acc = accuracy_score(y_train,xg_class.predict(X_train))
test_acc = accuracy_score(y_test,preds)
print(f'Train ACC: {train_acc}')
print(f'Test ACC: {test_acc}')

Train ACC: 1.0
Test ACC: 0.9629629629629629
